In [1]:
from transformers import *
import os
import pandas as pd
import tokenizers

I0518 17:27:21.938136 139812764055296 file_utils.py:41] PyTorch version 1.4.0 available.
I0518 17:27:23.113693 139812764055296 file_utils.py:57] TensorFlow version 2.1.0 available.


In [30]:
# model_type = "roberta-base"

# tokenizer =  RobertaTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
#             merges_file="transformers_vocab/{}-merges.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )

model_type = "albert-base-v2"

tokenizer = AlbertTokenizer.from_pretrained(
            pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
            lowercase=True,
        )

# model_type = "bert-base-cased"

# tokenizer =  BertTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )


def get_prediction_v3(tokenizer, tweet, selected_text):
    
    # lower cased here
    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())

    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets = []
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for word in tweet.split():
        
        sub_words = tokenizer.tokenize(" " + word)
        encoded_word = tokenizer.convert_tokens_to_ids(sub_words)
        orig_word = tokenizer.convert_tokens_to_string(sub_words)
#         print(" " + word, orig_word)
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word
        
        start_offsets = cursor
        cursor += len(" " + word)
        end_offsets = cursor
        
        token_level_cursor = start_offsets
        
        for i in range(number_of_tokens):
            
            tweet_offsets.append((start_offsets, end_offsets))
            
            # for bert tokenizer, replace "##" and add " " for first sub_word
            sub_word_len = len(sub_words[i].replace("##", ""))
            if i == 0 and orig_word != " " + word:
                sub_word_len += 1
            
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
            token_level_cursor += sub_word_len
        
#     print(tweet_offsets)
#     print(tweet_offsets_token_level)
    
    # get word idx
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
#         print(tweet[offset1: offset2])
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     print(targets_start, targets_end)
    
    
    return tweet[tweet_offsets[targets_start][0]: tweet_offsets[targets_end][1]], \
            tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]]

# tweet = "star wars ............ is **** boo??? i wanna do your job hand it over u can act as me at my high school lol"
# selected_text = "l"

tweet = "Sooo SAD I will miss you here in San Diego!!!"
selected_text = "sooo sad"

get_prediction_v3(tokenizer, tweet, selected_text)

(' sooo sad', ' sooo sad')

In [14]:
model_type = "roberta-base"

tokenizer_0 = RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tokenizer_1 = tokenizers.ByteLevelBPETokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tweet = " miles from you i`m in essex so give me plenty of warning so i can arrive in time to get at least one of those free beers."


input_ids_orig_0 = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(tweet))

ids_0 = []
offsets_0 = []
offsets_0_token_level = []
cursor = 0


for i, word in enumerate(tweet.lower().split()):
    
    encoded_word = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(" " + word))
    ids_0 += encoded_word
    
    number_of_tokens = len(encoded_word)

    start_offsets = cursor
    cursor += len(" " + word)
    end_offsets = cursor
    
    token_level_cursor = start_offsets

    for i in range(number_of_tokens):
        offsets_0.append((start_offsets, end_offsets))
        curr_token = tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(encoded_word[i]))
        
        offsets_0_token_level.append((token_level_cursor, token_level_cursor + len(curr_token)))
        token_level_cursor += len(curr_token)
    
# print(input_ids_orig_0)
# print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(input_ids_orig_0)))
print(ids_0)
print(offsets_0_token_level)
print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(ids_0)))

ids_1 = tokenizer_1.encode(tweet).ids
print(ids_1)
print(tokenizer_1.encode(tweet).offsets)
print(tokenizer_1.decode(ids_1))

[1788, 31, 47, 939, 12905, 119, 11, 2714, 8821, 98, 492, 162, 2710, 9, 2892, 98, 939, 64, 5240, 11, 86, 7, 120, 23, 513, 65, 9, 167, 481, 16328, 4]
[(0, 6), (6, 11), (11, 15), (15, 17), (17, 18), (18, 19), (19, 22), (22, 25), (25, 28), (28, 31), (31, 36), (36, 39), (39, 46), (46, 49), (49, 57), (57, 60), (60, 62), (62, 66), (66, 73), (73, 76), (76, 81), (81, 84), (84, 88), (88, 91), (91, 97), (97, 101), (101, 104), (104, 110), (110, 115), (115, 121), (121, 122)]
 miles from you i`m in essex so give me plenty of warning so i can arrive in time to get at least one of those free beers.
[1788, 31, 47, 939, 12905, 119, 11, 2714, 8821, 98, 492, 162, 2710, 9, 2892, 98, 939, 64, 5240, 11, 86, 7, 120, 23, 513, 65, 9, 167, 481, 16328, 4]
[(0, 6), (6, 11), (11, 15), (15, 17), (17, 18), (18, 19), (19, 22), (22, 25), (25, 28), (28, 31), (31, 36), (36, 39), (39, 46), (46, 49), (49, 57), (57, 60), (60, 62), (62, 66), (66, 73), (73, 76), (76, 81), (81, 84), (84, 88), (88, 91), (91, 97), (97, 101), (10

In [4]:
train = pd.read_csv("/media/jionie/my_disk/Kaggle/Tweet/input/tweet-sentiment-extraction/train.csv")

In [6]:
# model_type = "albert-base-v2"

# tokenizer = AlbertTokenizer.from_pretrained(
#             pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
#             lowercase=True,
#         )

model_type = "bert-base-cased"

tokenizer =  BertTokenizer(
            vocab_file="transformers_vocab/{}-vocab.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

for i in range(len(train)):
    
#     label = str(train["selected_text"].iloc[i]).lower().strip()
#     prediction, prediction_token = get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i])
#     prediction = prediction.strip()
#     prediction_token = prediction_token.strip()
#     print(prediction, "--------------------", prediction_token)
    if label != prediction:
        print(
              label, \
              "-------------------", \
              get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i]), \
              "-------------------", train["text"].iloc[i])

l ------------------- (' i`d have responded, if i were going', 'i`d have responded, if i were goin') -------------------  I`d have responded, if I were going
l ------------------- (' sooo sad', ' sooo sa') -------------------  Sooo SAD I will miss you here in San Diego!!!
l ------------------- (' bullying me...', ' bullying me') ------------------- my boss is bullying me...
l ------------------- (' leave me alone', ' leave me alon') -------------------  what interview! leave me alone
l ------------------- (' sons of ****,', ' sons of ****') -------------------  Sons of ****, why couldn`t they put them on the releases we already bought
l ------------------- (' http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth', ' http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on eart') ------------------- http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
l ------------------- (' fun

l ------------------- (' thunderstorms...perfect match', 'perfect matc') ------------------- 90 degrees, gross skies, and thunderstorms...perfect match for my mood  lol
l ------------------- (' aww thanks!', ' thanks') ------------------- _2nd aww thanks!
l ------------------- (' sorry', ' sorr') -------------------  Sorry RB is on PS3 for me
l ------------------- (' saw amazing heeels. but they were too big', ' saw amazing heeels. but they were too bi') ------------------- I saw amazing heeels. But they were too big
l ------------------- (' stuck', ' stuc') ------------------- I just stuck my finger down my throat and there are a bunch of bumps on my tongue & throat.
l ------------------- (' dang', ' dang ') ------------------- dang last url went down  ? http://blip.fm/~7aigm
l ------------------- (' missed all ur tweets', ' missed all ur tweets') -------------------  ohoh i missed all ur tweets  im gonna have to stay awake all night to see the announcement now, **** time difference
l

l ------------------- (' _what_ did you do? let me guess... you skipped sports and bought a new pinkish outfit?', '_what_ did you do? let me guess... you skipped sports and bought a new pinkish outfit') -------------------  _what_ did you do?  Let me guess... you skipped sports and bought a new pinkish outfit?
l ------------------- (' where is the rain please come out', ' where is the rain please come ou') ------------------- where is the rain please come out
l ------------------- (' my pleasure', ' my pleasur') -------------------  My pleasure
l ------------------- (' fml', 'fml') ------------------- 1st presentation for senior boards? FML
l ------------------- (' funny,', ' funny') -------------------  lol i know but it was just so funny, ahaha
l ------------------- (' grouchy', 'grouchy') ------------------- Wide awake and grouchy as ****.
l ------------------- (' worse.', ' worse') ------------------- just when i thought things couldn`t get any worse. it just did. and it will get e

l ------------------- (' calling mum too ask if she can by ice-cream', ' calling mum too ask if she can by ice-crea') ------------------- calling mum too ask if she can by ice-cream
l ------------------- (' having a very lazy day, playing xbox and drinking tea', ' having a very lazy day, playing xbox and drinking te') ------------------- _ having a very lazy day, playing xbox and drinking tea
l ------------------- (' thatï¿½s disappointing', 's disappointin') -------------------  Well thatï¿½s disappointing to hear.
l ------------------- (' hopefully', ' hopefull') -------------------  well if i use all my tweets will be the push to make me go to the shop. and by the time im back it will be over hopefully
l ------------------- (' my mom just texted me and told me that rodney was chasing fireflies in their backyard. awwwww i`m miss', ' my mom just texted me and told me that rodney was chasing fireflies in their backyard. awwwww i`m mis') ------------------- My mom just texted me and tol

l ------------------- (' diapers?? feeding? cooking? i don`t do any of that stuff!!! hahaha... yeah, right! sounds like a normal day for', ' diapers?? feeding? cooking? i don`t do any of that stuff!!! hahaha... yeah, right! sounds like a normal day fo') -------------------  diapers??  Feeding?  Cooking?  I don`t do any of that stuff!!!  Hahaha... yeah, right!  Sounds like a normal day for me.
l ------------------- (' http://twitpic.com/4bckp - that should answer part of your question', ' http://twitpic.com/4bckp - that should answer part of your questio') -------------------  http://twitpic.com/4bckp - that should answer part of your question
l ------------------- (' hope ya sleepin well guys', ' hope ya sleepin well guy') -------------------  hope ya sleepin well  guys still here
l ------------------- (' wasn`t feeling it', ' wasn`t feeling i') -------------------  I wasn`t feeling it
l ------------------- (' watching some vintage topgear', ' watching some vintage topgear') ----------

...

l ------------------- (' good', ' goo') -------------------  YES!!! one final party  better make it a good one !! how much mpre stuff you got to go?
l ------------------- (' would like to be able to go just to one of those', ' would like to be able to go just to one of thos') -------------------  I would like to be able to go just to ONE of those
l ------------------- (' woohoo..this is the day/week the lord has made! just remembered we get to wear jeans & chanclas all week!', 'woohoo..this is the day/week the lord has made! just remembered we get to wear jeans & chanclas all week') ------------------- Woohoo..this is the day/week the Lord has made! Just remembered we get to wear jeans & chanclas all week!
l ------------------- (' hello how are you', ' hello how are yo') -------------------  Hello How are you
l ------------------- (' this laptop is going to die,', ' this laptop is going to die') ------------------- this laptop is going to die, its not working!
l ------------------- (' 

l ------------------- (' water we sta focused we are on the job', ' water we sta focused we are on the jo') ------------------- Right now at my second gig !!! Sippinn on guess whut ? Water  we sta focused we are on the job
l ------------------- (' good', ' goo') ------------------- Had a good end to a busy day and now I`m off to bed with a full belly of sushi
l ------------------- (' yayay', ' yayay') ------------------- about to go to acting school  yayay
l ------------------- (' got a prerecorded phone call from jj asking to vote for admk led front, phone number is +914044621160, hyderabad connections', ' got a prerecorded phone call from jj asking to vote for admk led front, phone number is +914044621160, hyderabad connection') ------------------- got a prerecorded phone call from JJ asking to vote for ADMK led front, phone number is +914044621160, hyderabad connections
l ------------------- (' happy mother`s day', ' happy mother`s da') ------------------- happy mother`s day  & late

l ------------------- (' studying with and colette!', ' studying with and colette!') ------------------- studying with  and colette!
l ------------------- (' happy', ' happ') ------------------- wants to wish all the moms out there (and moms-to-be) a very happy Mother`s Day!
l ------------------- (' it`s ok lol i think we might do the same dates.', ' it`s ok lol i think we might do the same dates') -------------------  it`s ok lol  I think we might do the same dates.
l ------------------- (' smile~life is good!', ' smile~life is goo') ------------------- Smiling!  These guys are!    Turn up your speakers and SMILE~life is good! http://www.imtiredonline.com/smile/
l ------------------- (' this cold weather is killing me', ' this cold weather is killing m') ------------------- wants to move to QLD already, this cold weather is killing me
l ------------------- (' am sooooo mad rite now...it', ' am sooooo mad rite now..') ------------------- _mcsupergirl ok, finished set u free, and i am s

l ------------------- (' who`ve u noticed now kate ?', ' who`ve u noticed now kate ') -------------------  Who`ve u noticed now Kate ?
l ------------------- (' am little bit upset', ' am little bit upse') ------------------- am little bit upset   ppl is jerk-
l ------------------- (' she was wonderful, did not get to meet her though', ' she was wonderful, did not get to meet her thoug') -------------------  she was wonderful, did not get to meet her though
l ------------------- (' aww..', 'aww.. ') ------------------- Aww.. I lost 3 followers.  FOLLOW ME !
l ------------------- (' im going to my dance class now', ' im going to my dance class no') ------------------- _e im going to my dance class now
l ------------------- (' problem', ' proble') -------------------  yes .. thats a problem   #prism #fail
l ------------------- (' im not friends with anyone anymore', ' im not friends with anyone anymor') ------------------- Im not friends with anyone anymore
l ------------------- (' have f

IndexError: list index out of range

In [6]:
tokenizer.tokenize(tweet)

['▁see',
 '▁wat',
 '▁i',
 '▁mean',
 '▁bout',
 '▁fo',
 'll',
 '0',
 'w',
 '▁fri',
 'i',
 'day',
 's',
 '.',
 '.',
 '.',
 '▁it',
 '`',
 's',
 '▁called',
 '▁lose',
 '▁f',
 '0',
 'l',
 'lower',
 's',
 '▁friday',
 '.',
 '.',
 '.',
 '▁',
 's',
 'mh']

In [7]:
encoded = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet))
encoded

[196,
 15553,
 31,
 884,
 8244,
 4310,
 211,
 387,
 499,
 6266,
 49,
 1185,
 18,
 9,
 9,
 9,
 32,
 1,
 18,
 227,
 2926,
 398,
 387,
 255,
 17591,
 18,
 4619,
 9,
 9,
 9,
 13,
 18,
 17305]

In [8]:
decoded = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded))
decoded

'see wat i mean bout foll0w friidays... it<unk>s called lose f0llowers friday... smh'

In [9]:
print(len(encoded), len(tokenizer.tokenize(tweet)))

33 33


In [10]:
tweet.split()

['SEe',
 'waT',
 'I',
 'Mean',
 'bOuT',
 'FoLL0w',
 'fRiiDaYs...',
 'It`S',
 'cALLed',
 'LoSe',
 'f0LloWeRs',
 'FridAy...',
 'smH']

In [11]:
for (i, word) in enumerate(tweet.split()):
        
    sub_tokens = tokenizer.tokenize(word)

    print(sub_tokens)

['▁see']
['▁wat']
['▁i']
['▁mean']
['▁bout']
['▁fo', 'll', '0', 'w']
['▁fri', 'i', 'day', 's', '.', '.', '.']
['▁it', '`', 's']
['▁called']
['▁lose']
['▁f', '0', 'l', 'lower', 's']
['▁friday', '.', '.', '.']
['▁', 's', 'mh']
